In [1]:
import json
import logging

HOMEPAGE = "http://ieda.ust.hk/eng/events.php?catid=6&sid=50"
SITE = "http://ieda.ust.hk/eng/event_detail.php?type=E&id="

settings = None
try:
    with open("settings.json", "r") as f:
        settings = json.load(f)
except FileNotFoundError:
    logging.error("settings.json not found")

eventID = 805

import urllib.request
# contents = urllib.request.urlopen(SITE+str(eventID)).read()

from bs4 import BeautifulSoup
# soup = BeautifulSoup(contents, 'html.parser')

In [27]:
def scrapping(eventID):
  contents = urllib.request.urlopen(HOMEPAGE).read()
  soup = BeautifulSoup(contents, 'html.parser')

  # Get the latest event ID
  lastEventID = int(soup.find('div', class_='info-block__date active').attrs['data-tab'])

  if eventID > lastEventID:
    logging.info("No new event")
    return None

  contents = urllib.request.urlopen(SITE+str(eventID)).read()
  soup = BeautifulSoup(contents, 'html.parser')

  event = dict()

  eventType = soup.find(class_='context__subtitle').text.strip() #Joint OM/IE Seminar / Seminar / Thesis Examination / etc

  if eventType is None:
    logging.info("Invalid event")
    return None
  else:
    event['Type'] = eventType

  event["Title"] = soup.find(class_='context__title').text.strip() #Title
  
  for label, value in zip(soup.find_all('span', class_='item-label'), soup.find_all('span', class_='item-value')):
    event[label.text] = value.text #Date, Time, Venue, Speaker, etc, School is Optional

  event["Abstract"] = soup.find(class_='context__text abstract').text.strip() #Abstract

  speaker_bio = soup.find(class_='context__text speaker')
  if speaker_bio is not None:
    event["Bio"] = speaker_bio.text.strip() #Bio

  event["Link"] = SITE+str(eventID) #Link

  return event



In [28]:
event = scrapping(808)
event

{'Type': 'Joint IE/OM Seminar',
 'Title': 'Sixty Years of Applied Probability',
 'Date': '5 March 2024 (Tuesday)',
 'Time': '10:30 a.m.',
 'Venue': 'Room 2405 (lift 17-18)',
 'School': 'School of Business, Stanford University',
 'Speaker': 'Prof.  J. Michael HARRlSON , Adams Distinguished Professor',
 'Abstract': 'Abstract            \nAbout 60 years ago, the term "applied probability" (AP) became official when the Journal of Applied Probability was founded at the University of Sheffield. In the years that followed, both the Operations Research Society of America and the Institute for Mathematical Statistics formally designated AP as a subfield of their respective disciplines. By coincidence, my first exposure to an AP model (specifically, the M/M/1 queue) also came about 60 years ago, as an undergraduate engineering student. In this lecture I will look back on developments in the field, in some which I have participated, with particular emphasis on (a) how to define and characterize A

In [32]:
description = event["Title"] + "\n\nBy " + event["Speaker"]
if "School" in event:
    description += "\nFrom " + event["School"]
description +=   "\n\n" + event["Link"] + "\n\n" + event["Abstract"] + "\n\n" + event["Bio"]


print(description)

Sixty Years of Applied Probability

By Prof.  J. Michael HARRlSON , Adams Distinguished Professor
From School of Business, Stanford University

Abstract            
About 60 years ago, the term "applied probability" (AP) became official when the Journal of Applied Probability was founded at the University of Sheffield. In the years that followed, both the Operations Research Society of America and the Institute for Mathematical Statistics formally designated AP as a subfield of their respective disciplines. By coincidence, my first exposure to an AP model (specifically, the M/M/1 queue) also came about 60 years ago, as an undergraduate engineering student. In this lecture I will look back on developments in the field, in some which I have participated, with particular emphasis on (a) how to define and characterize AP, and (b) shining jewels of the AP literature.

Speaker Biography            
J. MICHAEL HARRlSON is the Adams Distinguished Professor of Management, Emeritus, in the Gradu

In [24]:
import datetime
# event["Date"]
# # datetime.datetime.strptime(event["Date"] + " " + event["Time"], "%d %B %Y (%A) %I:%M %p")
startTime = datetime.datetime.strptime(event["Date"] + " " + event["Time"].replace(".", "").capitalize(), "%d %B %Y (%A) %I:%M %p")
endTime = startTime + datetime.timedelta(hours=1.5)
endTime.isoformat()


'2024-03-05T12:00:00'

In [24]:
title = soup.find(class_='context__text speaker').text.strip()
title

'Speaker Biography            \nProf. Tingliang Huang is a Professor and the Amazon Distinguished Professor of Business Analytics at the Haslam College of Business, University of Tennessee (UT), and the Business Analytics PhD Program Recruiting Lead. He is also an Honorary Professor at UCL School of Management, University College London (UCL), the United Kingdom. He is an award-winning scholar & educator. He is interested in business analytics, AI, data science, new business models, operations-marketing interface, supply chain management, service operations, innovation, and socially responsible operations. His research articles have been published in top business journals such as Manufacturing & Service Operations Management, Marketing Science, Management Science, and Production and Operations Management. He has won various research & teaching awards including the 2023 INFORMS Workshop on Data Science Best Paper Award, 2018 POMS Wickham Skinner Early Career Research Accomplishments Awa

In [25]:
soup.find_all('span', class_='item-label')


[<span class="item-label">Date</span>,
 <span class="item-label">Time</span>,
 <span class="item-label">Venue</span>,
 <span class="item-label">Speaker</span>]

In [26]:
event = dict()

soup.find_all('span', class_='item-value')

for label, value in zip(soup.find_all('span', class_='item-label'), soup.find_all('span', class_='item-value')):
    print(label.text, value.text)
    event[label.text] = value.text

Date 23 February 2024 (Friday)
Time 10:30 am
Venue LSK - Room 4047
Speaker Prof. Tingliang HUANG , Distinguished Professor


In [5]:
HOMEPAGE = "http://ieda.ust.hk/eng/events.php?catid=6&sid=50"

contents_home = urllib.request.urlopen(HOMEPAGE).read()
soup2 = BeautifulSoup(contents_home, 'html.parser')

In [6]:
int(soup2.find('div', class_='info-block__date active').attrs['data-tab'])

811

In [7]:
soup.find(class_='context__date').text.strip()

AttributeError: 'NoneType' object has no attribute 'text'